In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
import lightning as pl
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, transforms
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR, CosineAnnealingLR
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torchmetrics import Accuracy
import torchvision.models as models
from lightning.pytorch import Callback

# Optimizers and schedulars class
### *here I have written a class which consists of some optimizers and some schedulars to make it easy to combine them together and use in model*

In [2]:
class OptimizerSchedulerFactory:
    def __init__(self, model_parameters, optimizer_name="adam", lr=1e-4, scheduler_name=None):
        self.model_parameters = model_parameters
        self.optimizer_name = optimizer_name.lower()
        self.lr = lr
        self.scheduler_name = scheduler_name.lower() if scheduler_name else None

    def get_optimizer(self):
        if self.optimizer_name == "adam":
            optimizer = Adam(self.model_parameters, lr=self.lr)
        elif self.optimizer_name == "sgd":
            optimizer = SGD(self.model_parameters, lr=self.lr, momentum=0.9)
        elif self.optimizer_name == "adamw":
            optimizer = AdamW(self.model_parameters, lr=self.lr)
        else:
            raise ValueError(f"Unsupported optimizer: {self.optimizer_name}")
        return optimizer

    def get_scheduler(self, optimizer):
        if self.scheduler_name is None:
            return None
        elif self.scheduler_name == "reduce_on_plateau":
            scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=5)
        elif self.scheduler_name == "step_lr":
            scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
        elif self.scheduler_name == "cosine_annealing":
            scheduler = CosineAnnealingLR(optimizer, T_max=50)
        else:
            raise ValueError(f"Unsupported scheduler: {self.scheduler_name}")
        return scheduler

    def get_optimizer_scheduler(self):
        optimizer = self.get_optimizer()
        scheduler = self.get_scheduler(optimizer)
        return optimizer, scheduler


# Model
### *Here I write the model based on it's structure in the article*

In [3]:
class SEBlock1D(nn.Module):
    def __init__(self, input_dim=None, reduction_ratio=4):
        super(SEBlock1D, self).__init__()
        self.input_dim = input_dim
        self.reduction_ratio = reduction_ratio
        self.fc1 = None
        self.fc2 = None  
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def _initialize_fc_layers(self, input_dim):
        reduced_dim = max(1, input_dim // self.reduction_ratio)  
        self.fc1 = nn.Linear(input_dim, reduced_dim) 
        self.fc2 = nn.Linear(reduced_dim, input_dim)
    
    def forward(self, x):
        if self.fc1 is None or self.fc2 is None:
            input_dim = x.size(-1) 
            self._initialize_fc_layers(input_dim)
        pooled = x.mean(dim=1) 
        squeezed = self.fc1(pooled)
        activated = self.relu(squeezed)
        excited = self.fc2(activated)
        scaling_weights = self.sigmoid(excited)
        scaling_weights = scaling_weights.unsqueeze(1) 
        scaled_output = x * scaling_weights
        return scaled_output
# class SeparableConv1D(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size, padding='same', activation=nn.ReLU()):

#         super(SeparableConv1D, self).__init__()

#         self.depthwise = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size,  padding = 'same')
#         self.activation = activation

#     def forward(self, x):
#         x = self.depthwise(x)
#         if self.activation:
#             x = self.activation(x)
#         return x


class SeparableConv1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding='same', activation=nn.ReLU()):
        super(SeparableConv1D, self).__init__()
        self.depthwise = nn.Conv1d(
            in_channels=in_channels,
            out_channels=in_channels,  
            kernel_size=kernel_size,
            padding=padding,
            groups=in_channels
        )
        self.pointwise = nn.Conv1d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=1
        )
        self.activation = activation

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        if self.activation:
            x = self.activation(x)
        return x

class ConcatBlockWithSEBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, ratio=8, pool_size=2):
        super(ConcatBlockWithSEBlock1D, self).__init__()
        self.sepconv1 = SeparableConv1D(in_channels, out_channels, kernel_size=3, activation=nn.ReLU())  # Conv1
        self.sepconv2 = SeparableConv1D(out_channels, out_channels, kernel_size=3, activation=nn.ReLU())  # Conv2
        self.seblock = SEBlock1D(input_dim=None, reduction_ratio=ratio)  # SE Block
        self.maxpool = nn.MaxPool1d(kernel_size=pool_size)  # MaxPooling

    def forward(self, x):
        residual = x  # Save input for concatenation
        out = self.sepconv1(x)  # First convolution
        out = self.sepconv2(out)  # Second convolution
        concat = torch.cat([residual, out], dim=1)  # Concatenate input and output
        se_output = self.seblock(concat)  # Pass through SE Block
        pooled_output = self.maxpool(se_output)  # MaxPooling
        return pooled_output




class FinalShallowModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super(FinalShallowModel, self).__init__()
        self.learning_rate = learning_rate

        # Initial convolution layer
        self.initial_conv = SeparableConv1D(in_channels=2, out_channels=16, kernel_size=5, activation=nn.ReLU())
        
        # Concat blocks with input-output dimensions properly aligned
        self.block1 = ConcatBlockWithSEBlock1D(in_channels=16, out_channels=16, ratio=8, pool_size=4)  # Input 16 -> Output 16
        self.block2 = ConcatBlockWithSEBlock1D(in_channels=32, out_channels=16, ratio=8, pool_size=4)  # Input 32 -> Output 24
        self.block3 = ConcatBlockWithSEBlock1D(in_channels=48, out_channels=16, ratio=8, pool_size=4)  # Input 48 -> Output 32
        self.block4 = ConcatBlockWithSEBlock1D(in_channels=64, out_channels=16, ratio=8, pool_size=1)  # Input 64 -> Output 40

        # Final layers
        self.global_max_pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(80, 2)  # Final FC with concatenated dimensions
        self.softmax = nn.Softmax(dim=1)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.initial_conv(x)  # Initial convolution
        x = self.block1(x)        # Block 1
        x = self.block2(x)        # Block 2
        x = self.block3(x)        # Block 3
        x = self.block4(x)        # Block 4
        x = self.global_max_pool(x).squeeze(-1)  # Global pooling
        x = self.fc(x)            # Fully connected layer
        return x
    def predict(self, x):
        logits = self.forward(x)
        probabilities = self.softmax(logits)
        return probabilities
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [4]:
from torch.utils.data import DataLoader, TensorDataset
X = torch.randn(100, 2, 1024)  # 100 samples, 2 channels, 1024 length
y = torch.randint(0, 2, (100,))  # Binary classification (0 or 1)
train_dataset = TensorDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
model = FinalShallowModel(learning_rate=1e-3)
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    logger=False 
)
trainer.fit(model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory c:\Users\asus\Desktop\Company\Candidate Models\AMC\Shallow\checkpoints exists and is not empty.

  | Name            | Type                     | Params | Mode 
---------------------------------------------------------------------
0 | initial_conv    | SeparableConv1D          | 60     | train
1 | block1          | ConcatBlockWithSEBlock1D | 672    | train
2 | block2          | ConcatBlockWithSEBlock1D | 992    | train
3 | block3          | ConcatBlockWithSEBlock1D | 1.3 K  | train
4 | block4          | ConcatBlockWithSEBlock1D

Epoch 0:   0%|          | 0/7 [00:00<?, ?it/s] 

c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\core\module.py:516: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Epoch 9: 100%|██████████| 7/7 [00:00<00:00, 18.31it/s, train_loss=0.695]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 7/7 [00:00<00:00, 17.19it/s, train_loss=0.695]
